In [3]:
import geopandas as gpd
import fiona
import numpy as np

In [4]:
#read in the community and Cta bus and trian stops data as GeoDataFrames
community_areas = gpd.read_file('../Converted/Chicago_Community_Area_Boundaries-polygon.shp')
cta_bus_stops = gpd.read_file('../Converted/CTA_BusStops-point.shp')
cta_train_stops = gpd.read_file('../Converted/CTA_RailStations-point.shp')

In [5]:
#perform a spatial join to associated each CTA stop with its corresponding community area
cta_bus_stops_by_community = gpd.sjoin(cta_bus_stops, community_areas, predicate="within")
cta_bus_stops_by_community


,Name,Field_1,SYSTEMSTOP,STREET,CROSS_ST,DIR,POS,ROUTESSTPG,CITY,STATUS,...,index_right,area,area_num_1,area_numbe,comarea,comarea_id,community,perimeter,shape_area,shape_len
0,East River Rd & Berwyn,East River Rd &amp; Berwyn,8590,EAST RIVER RD,BERWYN,NB,NS,81W,CHICAGO,In Service,...,74,0,76,76,0,0,OHARE,0,3.718356e+08,173625.984660
1,Cumberland Blue Line Station,Cumberland Blue Line Station,14143,CUMBERLAND,BLUE LINE STATION,SWB,TERM,81W,CHICAGO,In Service,...,74,0,76,76,0,0,OHARE,0,3.718356e+08,173625.984660
1397,Lawrence & East River Rd,Lawrence &amp; East River Rd,8581,LAWRENCE,EAST RIVER RD,EB,FS,81W,CHICAGO,In Service,...,74,0,76,76,0,0,OHARE,0,3.718356e+08,173625.984660
1398,East River Rd & Argyle,East River Rd &amp; Argyle,8585,EAST RIVER RD,ARGYLE,SB,NT,81W,CHICAGO,In Service,...,74,0,76,76,0,0,OHARE,0,3.718356e+08,173625.984660
1399,East River Rd & Evelyn Lane,East River Rd &amp; Evelyn Lane,8592,EAST RIVER RD,EVELYN LANE,NB,NS,81W,CHICAGO,In Service,...,74,0,76,76,0,0,OHARE,0,3.718356e+08,173625.984660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9383,43rd Street & Ellis,43rd Street &amp; Ellis,7994,43RD STREET,ELLIS,WB,NS,43,CHICAGO,In Service,...,1,0,36,36,0,0,OAKLAND,0,1.691396e+07,19565.506153
9384,42nd Place & Lake Park,42nd Place &amp; Lake Park,7991,42ND PLACE,LAKE PARK,WB,NS,43,CHICAGO,In Service,...,1,0,36,36,0,0,OAKLAND,0,1.691396e+07,19565.506153
9385,Lake Park & 40th Street,Lake Park &amp; 40th Street,7743,LAKE PARK,40TH STREET,NB,FS,39,CHICAGO,In Service,...,1,0,36,36,0,0,OAKLAND,0,1.691396e+07,19565.506153
10685,43rd Street & Cottage Grove,43rd Street &amp; Cottage Grove,7997,43RD STREET,COTTAGE GROVE,WB,NS,43,CHICAGO,In Service,...,1,0,36,36,0,0,OAKLAND,0,1.691396e+07,19565.506153


In [6]:
cta_bus_stops_by_community.columns

Index(['Name', 'Field_1', 'SYSTEMSTOP', 'STREET', 'CROSS_ST', 'DIR', 'POS',
       'ROUTESSTPG', 'CITY', 'STATUS', 'PUBLIC_NAM', 'POINT_X', 'POINT_Y',
       'SHAPE', 'OWLROUTES', 'geometry', 'index_right', 'area', 'area_num_1',
       'area_numbe', 'comarea', 'comarea_id', 'community', 'perimeter',
       'shape_area', 'shape_len'],
      dtype='object')

In [7]:
cta_bus_stops_by_community.area_numbe

0        76
1        76
1397     76
1398     76
1399     76
         ..
9383     36
9384     36
9385     36
10685    36
10686    36
Name: area_numbe, Length: 9959, dtype: int64

In [8]:
#count the number of CTA bus stops in each community area
bus_stops_community = cta_bus_stops_by_community.groupby(['area_numbe']).size().reset_index(name='bus_count')

In [9]:
bus_stops_community

,area_numbe,bus_count
0,1,97
1,2,196
2,3,175
3,4,132
4,5,113
...,...,...
72,73,115
73,74,82
74,75,88
75,76,30


In [10]:
#perform a spatial join to associated each CTA stop with its corresponding community area
cta_train_stops_by_community = gpd.sjoin(cta_train_stops, community_areas, predicate="within")
cta_train_stops_by_community.head()

,Name,Field_1,Station ID,Station Na,Rail Line,Address,ADA,Park and R,POINT_X,POINT_Y,...,index_right,area,area_num_1,area_numbe,comarea,comarea_id,community,perimeter,shape_area,shape_len
2,Pulaski,Pulaski,150,Pulaski-Douglas,Pink,2005 S. Pulaski Road,ADA Accessible,0,1.150164e+06,1.889924e+06,...,29,0,29,29,0,0,NORTH LAWNDALE,0,8.948742e+07,44959.459663
25,Kostner,Kostner,600,Kostner,Pink,2019 S. Kostner Avenue,ADA Accessible,0,1.147730e+06,1.889868e+06,...,29,0,29,29,0,0,NORTH LAWNDALE,0,8.948742e+07,44959.459663
77,Central Park,Central Park,780,Central Park,Pink,1915 S. Central Park Avenue,ADA Accessible,0,1.152738e+06,1.889992e+06,...,29,0,29,29,0,0,NORTH LAWNDALE,0,8.948742e+07,44959.459663
105,Kedzie,Kedzie,1040,Kedzie-Douglas,Pink,1944 S. Kedzie Avenue,ADA Accessible,0,1.155312e+06,1.890061e+06,...,29,0,29,29,0,0,NORTH LAWNDALE,0,8.948742e+07,44959.459663
3,Logan Square,Logan Square,1020,Logan Square,Blue Line,2620 N. Milwaukee Av,ADA Accessible,0,1.154264e+06,1.917602e+06,...,22,0,22,22,0,0,LOGAN SQUARE,0,1.000576e+08,49213.421749


In [11]:
cta_train_stops_by_community.columns

Index(['Name', 'Field_1', 'Station ID', 'Station Na', 'Rail Line', 'Address',
       'ADA', 'Park and R', 'POINT_X', 'POINT_Y', 'geometry', 'index_right',
       'area', 'area_num_1', 'area_numbe', 'comarea', 'comarea_id',
       'community', 'perimeter', 'shape_area', 'shape_len'],
      dtype='object')

In [12]:
#count the number of CTA train stops in each community area
train_stops_community = cta_train_stops_by_community.groupby(['area_numbe']).size().reset_index(name='train_count')
train_stops_community

,area_numbe,train_count
0,1,4
1,3,3
2,4,4
3,5,2
4,6,7
5,7,2
6,8,7
7,10,1
8,11,1
9,14,3


In [13]:
total_stops = bus_stops_community.copy()

In [14]:
total_stops = total_stops.merge(train_stops_community, how = 'left', on='area_numbe')
total_stops

,area_numbe,bus_count,train_count
0,1,97,4.0
1,2,196,NaN
2,3,175,3.0
3,4,132,4.0
4,5,113,2.0
...,...,...,...
72,73,115,NaN
73,74,82,NaN
74,75,88,NaN
75,76,30,2.0


In [15]:
total_stops.fillna(0)

,area_numbe,bus_count,train_count
0,1,97,4.0
1,2,196,0.0
2,3,175,3.0
3,4,132,4.0
4,5,113,2.0
...,...,...,...
72,73,115,0.0
73,74,82,0.0
74,75,88,0.0
75,76,30,2.0


In [16]:
total_stops['Total_Stops'] = total_stops['bus_count'] + total_stops['train_count']
total_stops

,area_numbe,bus_count,train_count,Total_Stops
0,1,97,4.0,101.0
1,2,196,NaN,NaN
2,3,175,3.0,178.0
3,4,132,4.0,136.0
4,5,113,2.0,115.0
...,...,...,...,...
72,73,115,NaN,NaN
73,74,82,NaN,NaN
74,75,88,NaN,NaN
75,76,30,2.0,32.0


In [17]:
total_stops.to_csv('stop_data.csv')